# Møller-Plesset 微扰， MP2

## 极简MP2关联能计算
MP2计算的起点是HF，我们先做一个HF

In [5]:
from pyscf import gto, scf
import numpy as np

def geometry_h2o(bl = 1.0 , theta = np.pi *104/180):
    """get geometry for h2o"""
    geometry = []
    geometry.append(("O",(0,0,0)))
    geometry.append(("H",(0,bl*np.sin(theta/2),bl*np.cos(theta/2))))
    geometry.append(("H",(0,-bl*np.sin(theta/2),bl*np.cos(theta/2))))
    return geometry

def geometry_ch4():
    """get geometry for ch4"""
    geometry = []
    geometry.append(("C",(0,0,0.0)))
    geometry.append(("H",(    0.554102923 , 0.799604215 ,    0.496502616)))
    geometry.append(("H",( 0.683303601 , -0.813404287 ,   -0.253601339)))
    geometry.append(("H",(  -0.778204102, -0.37350197  ,   0.669203526)))
    geometry.append(("H",(  -0.459302421, 0.387402041   ,  -0.912104808)))
    return geometry

mol = gto.Mole()
mol.atom = geometry_ch4()
mol.basis = "cc-pvdz"
mol.build()

mf = scf.RHF(mol)
mf.run()

converged SCF energy = -40.1987064719646


一些必要的变量定义

In [6]:
from functools import reduce
from pyscf import ao2mo

norb = mol.nao_nr()   # 轨道数
nelec = mol.nelectron   # 电子数
assert (nelec % 2 == 0) # 仅考虑close shell
nocc = nelec //2       # 占据轨道数
nvir = norb - nocc       # 空轨道数
e_hf = mf.e_tot          # Hartree Fock 能

mo_coeff = mf.mo_coeff   # 分子轨道系数（用于原子轨道-分子轨道的变换）
mo_energy = mf.mo_energy  # 轨道能

h1e_mo = reduce(np.dot, (mo_coeff.T, mf.get_hcore(), mo_coeff))   # 分子轨道下的单电子积分
# eri = ao2mo.full(mf.mol, mo_coeff)
eri_mo = ao2mo.kernel(mf.mol, mo_coeff,aosym=1)
eri_mo = eri_mo.reshape((norb,norb,norb,norb))   # 分子轨道下的双电子积分

MP2相关能计算：(Modern Quantum Chemistry Eq. 6.74)
$$
E_{\mathrm{MP}2}=2 \sum_{i j a b}^{N / 2} \frac{\langle i j \mid a b\rangle\langle a b \mid i j\rangle}{\varepsilon_i+\varepsilon_j-\varepsilon_a-\varepsilon_b}-\sum_{i j a b}^{N / 2} \frac{\langle i j \mid a b\rangle\langle a b \mid j i\rangle}{\varepsilon_i+\varepsilon_j-\varepsilon_a-\varepsilon_b}
$$

$$
E_{\mathrm{MP} 2}= \sum_{i,j \in occ} \sum_{a,b \in vir} \frac{\langle i j \mid a b \rangle (2 \langle i j \mid a b \rangle- \langle i j \mid b a \rangle )}{\varepsilon_i + \varepsilon_j - \varepsilon_a-\varepsilon_b}
$$

In [7]:
emp2 = 0.0
for i in np.arange(nocc):
    for j in np.arange(nocc):
        for k in np.arange(nocc,norb):
            for l in np.arange(nocc,norb):
                emp2 += eri_mo[i,k,j,l]*(2*eri_mo[i,k,j,l] - eri_mo[i,l,j,k])/(mo_energy[i] + mo_energy[j] - mo_energy[k] - mo_energy[l])
emp2

-0.16421767323888858

与PySCF计算的MP2结果对比

In [8]:
from pyscf import mp
mymp = mp.mp2.MP2(mf)
emp2_pyscf ,_ = mymp.kernel()
emp2_pyscf

E(MP2) = -40.3629241452035  E_corr = -0.164217673238889
E(SCS-MP2) = -40.3687537290159  E_corr = -0.170047257051282


-0.16421767323888886

In [11]:
emp2 - emp2_pyscf

2.7755575615628914e-16

MP2的总能量为

In [9]:
mp2_total_energy = e_hf + emp2
mp2_total_energy

-40.36292414520347

In [10]:
mp2_total_energy - mymp.e_tot

0.0